In [1]:
import numpy as numpy
import pandas as pd
import os

In [2]:
period_order =[35,36,40,39,42,43,41,46,47,44,49,50,48,53,54,51,52,56,55,58,59,57,61]
all_period_order = [45,15,16,17,18,12,19,13,14,2,4,3,5,6,7,8,9,10,11,20,21,22,1,24,26,23,25,28,27,30,31,29,33,34,32,37,38,35,36,40,39,42,43,41,46,47,44,49,50,48,53,54,51,52,56,55,58,59,57,61,60]
period_order_name=["2017-2018A","2017-2018B","2017-2018V","2018-2019A","2018-2019B","2018-2019V","2019-2020A","2019-2020B","2019-2020V","2020-2021A","2020-2021B","2020-2021V","2021-2022A","2021-2022B","2021-2022V","2022-2023A","2022-2023B","2022-2023V","2023-2024A","2023-2024B","2023-2024V","2024-2025A","2024-2025B"]

In [3]:
def load_period_csvs_into_vars(base_directory):
    """
    Carga los archivos CSV de calificaciones organizados por periodo y carrera
    en variables globales con el formato p<periodo>.
    Cada variable p<periodo> será un diccionario, donde las claves son los nombres
    de los archivos CSV (con el formato p<periodo>c<carrera>s<semestre>) y los
    valores son DataFrames de pandas.

    Args:
        base_directory (str): La ruta base donde se encuentran las carpetas de 'Periodo_XX'.

    Returns:
        dict: Un diccionario donde cada clave es 'p<periodo>' y su valor es otro
              diccionario que contiene los DataFrames de los CSVs de ese período.
              Retorna un diccionario vacío si no se encuentran archivos.
    """
    period_data = {}

    if not os.path.exists(base_directory):
        print(f"Error: El directorio base '{base_directory}' no existe.")
        return period_data

    # Iterate over Periodo folders
    for period_folder_name in os.listdir(base_directory):
        if period_folder_name.startswith("Periodo_"):
            period_id = period_folder_name.replace("Periodo_", "")
            period_path = os.path.join(base_directory, period_folder_name)

            if os.path.isdir(period_path):
                current_period_dfs = {}

                # Iterate over Carrera folders within the period folder
                for career_folder_name in os.listdir(period_path):
                    if career_folder_name.startswith("Carrera_"):
                        career_path = os.path.join(period_path, career_folder_name)

                        if os.path.isdir(career_path):
                            for file_name in os.listdir(career_path):
                                if file_name.endswith(".csv"):
                                    csv_path = os.path.join(career_path, file_name)

                                    # Extract components for the new concise name
                                    # Example: periodo58carrera2semestre2.csv
                                    # -> p58c2s2
                                    parts = os.path.splitext(file_name)[0].split('periodo')[1].split('carrera')
                                    period_part = parts[0]
                                    career_sem_parts = parts[1].split('semestre')
                                    career_part = career_sem_parts[0]
                                    semestre_part = career_sem_parts[1]

                                    df_name = f"p{period_part}c{career_part}s{semestre_part}"

                                    try:
                                        df = pd.read_csv(csv_path)
                                        current_period_dfs[df_name] = df
                                        print(f"Cargado: {file_name} en p{period_id}['{df_name}']")
                                    except Exception as e:
                                        print(f"Error al cargar {file_name}: {e}")

                period_var_name = f"p{period_id}"
                period_data[period_var_name] = current_period_dfs
    
    if not period_data:
        print(f"No se encontraron datos de períodos en '{base_directory}'.")
        
    return period_data

In [ ]:
period_map = {id_p: name for id_p, name in zip(period_order, period_order_name)}
period_type = {id_p: name[-1] for id_p, name in zip(period_order, period_order_name)} # A, B, o V

csv_base_folder = r"D:/TesisDB/CSV's"

# --- Cargar todos los CSVs ---
print(f"Cargando datos desde: {csv_base_folder}")
all_period_data = load_period_csvs_into_vars(csv_base_folder)

if not all_period_data:
    print("No se pudieron cargar los datos. Asegúrate de que los CSVs existan en la ruta especificada.")
    exit()

print("Datos cargados exitosamente.")

# --- Diccionario para almacenar los nombres de las carreras (para mejor legibilidad) ---
# En un escenario real, esto se obtendría de la tabla nes_carreras de la base de datos.
# Por ahora, usaremos un mock-up o lo inferiremos si es posible.
# Si tienes una manera de obtener los nombres reales de las carreras, ¡sería ideal!
# Por ejemplo: career_names = {1: "Ing. Sistemas", 2: "Lic. Administracion", ...}
# Para este análisis, usaremos los id_carrera directamente, o podríamos inferir de los nombres de los archivos.
# Vamos a crear un placeholder para nombres de carrera si no los tenemos
career_names = {}
# Podríamos intentar inferir los IDs de carrera existentes y darles un nombre genérico
# Esto es solo un ejemplo, podrías querer cargar los nombres reales desde la DB
for p_key, p_val in all_period_data.items():
    for df_key in p_val.keys():
        try:
            # df_key looks like p<period>c<carrera>s<semestre>
            carrera_id = int(df_key.split('c')[1].split('s')[0])
            if carrera_id not in career_names:
                career_names[carrera_id] = f"Carrera {carrera_id}"
        except (IndexError, ValueError):
            pass # Ignore malformed keys

print(f"\nCarreras detectadas: {career_names}")

# --- Funciones de Análisis ---

def analyze_first_sem_population(data, p_map, p_type, c_names):
    """
    Analiza la población de primer semestre por período (solo tipo 'A') y carrera.
    """
    print("\n" + "="*50)
    print("Análisis de Población en PRIMER SEMESTRE (Periodos 'A')")
    print("="*50)

    results = []
    
    # Filtrar periodos tipo 'A' y ordenarlos según period_order
    a_periods = [p_id for p_id in period_order if p_type.get(p_id) == 'A']

    for p_id in a_periods:
        period_key = f"p{p_id}"
        period_name = p_map.get(p_id, f"Periodo {p_id}")

        if period_key in data:
            print(f"\n--- {period_name} (ID: {p_id}) ---")
            period_dfs = data[period_key]
            
            # Recopilar todos los DataFrames de semestre 1 para este período
            sem1_dfs = {df_name: df for df_name, df in period_dfs.items() if df_name.endswith('s1')}

            if sem1_dfs:
                for df_name, df in sem1_dfs.items():
                    try:
                        carrera_id = int(df_name.split('c')[1].split('s')[0])
                        carrera_name = c_names.get(carrera_id, f"Carrera {carrera_id}")
                        num_students = df['matricula_hash'].nunique() # Contar matrículas únicas
                        print(f"  {carrera_name} (ID: {carrera_id}): {num_students} estudiantes en 1er semestre.")
                        results.append({
                            'Periodo_ID': p_id,
                            'Periodo_Nombre': period_name,
                            'Carrera_ID': carrera_id,
                            'Carrera_Nombre': carrera_name,
                            'Semestre': 1,
                            'Num_Estudiantes': num_students
                        })
                    except (IndexError, ValueError):
                        print(f"  Advertencia: No se pudo parsear la ID de carrera de {df_name}.")
            else:
                print(f"  No se encontraron DataFrames para el 1er semestre en este período.")
        else:
            print(f"  Datos no disponibles para el período {period_name} (ID: {p_id}).")
    
    if not results:
        print("No se encontraron datos para el 1er semestre en periodos 'A'.")
    else:
        first_sem_df = pd.DataFrame(results)
        print("\n--- Resumen General de Primer Semestre ---")
        print(first_sem_df.pivot_table(index='Carrera_Nombre', columns='Periodo_Nombre', values='Num_Estudiantes', fill_value=0))
        return first_sem_df


def analyze_tenth_sem_population(data, p_map, p_type, c_names):
    """
    Analiza la población de décimo semestre por período (solo tipo 'B') y carrera.
    """
    print("\n" + "="*50)
    print("Análisis de Población en DÉCIMO SEMESTRE (Periodos 'B')")
    print("="*50)

    results = []
    
    # Filtrar periodos tipo 'B' y ordenarlos según period_order
    b_periods = [p_id for p_id in period_order if p_type.get(p_id) == 'B']

    for p_id in b_periods:
        period_key = f"p{p_id}"
        period_name = p_map.get(p_id, f"Periodo {p_id}")

        if period_key in data:
            print(f"\n--- {period_name} (ID: {p_id}) ---")
            period_dfs = data[period_key]
            
            # Recopilar todos los DataFrames de semestre 10 para este período
            sem10_dfs = {df_name: df for df_name, df in period_dfs.items() if df_name.endswith('s10')}

            if sem10_dfs:
                for df_name, df in sem10_dfs.items():
                    try:
                        carrera_id = int(df_name.split('c')[1].split('s')[0])
                        carrera_name = c_names.get(carrera_id, f"Carrera {carrera_id}")
                        num_students = df['matricula_hash'].nunique() # Contar matrículas únicas
                        print(f"  {carrera_name} (ID: {carrera_id}): {num_students} estudiantes en 10mo semestre.")
                        results.append({
                            'Periodo_ID': p_id,
                            'Periodo_Nombre': period_name,
                            'Carrera_ID': carrera_id,
                            'Carrera_Nombre': carrera_name,
                            'Semestre': 10,
                            'Num_Estudiantes': num_students
                        })
                    except (IndexError, ValueError):
                        print(f"  Advertencia: No se pudo parsear la ID de carrera de {df_name}.")
            else:
                print(f"  No se encontraron DataFrames para el 10mo semestre en este período.")
        else:
            print(f"  Datos no disponibles para el período {period_name} (ID: {p_id}).")
            
    if not results:
        print("No se encontraron datos para el 10mo semestre en periodos 'B'.")
    else:
        tenth_sem_df = pd.DataFrame(results)
        print("\n--- Resumen General de Décimo Semestre ---")
        print(tenth_sem_df.pivot_table(index='Carrera_Nombre', columns='Periodo_Nombre', values='Num_Estudiantes', fill_value=0))
        return tenth_sem_df


def exploratory_population_analysis(data, p_map, p_type, c_names):
    """
    Realiza un análisis exploratorio general de la población por período, carrera y semestre.
    """
    print("\n" + "="*50)
    print("Análisis Exploratorio General de Población")
    print("="*50)

    all_students_data = []

    for p_id in period_order:
        period_key = f"p{p_id}"
        period_name = p_map.get(p_id, f"Periodo {p_id}")
        current_period_type = p_type.get(p_id, 'N/A')

        if period_key in data:
            period_dfs = data[period_key]
            for df_name, df in period_dfs.items():
                try:
                    # Parsear id_carrera y semestre del nombre del DataFrame
                    carrera_id = int(df_name.split('c')[1].split('s')[0])
                    semestre_id = int(df_name.split('s')[1])
                    carrera_name = c_names.get(carrera_id, f"Carrera {carrera_id}")
                    num_students = df['matricula_hash'].nunique()

                    all_students_data.append({
                        'Periodo_ID': p_id,
                        'Periodo_Nombre': period_name,
                        'Tipo_Periodo': current_period_type,
                        'Carrera_ID': carrera_id,
                        'Carrera_Nombre': carrera_name,
                        'Semestre': semestre_id,
                        'Num_Estudiantes': num_students
                    })
                except (IndexError, ValueError):
                    print(f"  Advertencia: No se pudo parsear la ID de carrera o semestre de {df_name}.")
        # else:
            # print(f"  Datos no disponibles para el período {period_name} (ID: {p_id}).") # Descomentar si quieres ver los periodos sin datos

    if not all_students_data:
        print("No se encontraron datos para el análisis exploratorio general.")
        return None

    full_df = pd.DataFrame(all_students_data)

    print("\n--- Total de Estudiantes por Periodo y Carrera ---")
    total_by_period_career = full_df.groupby(['Periodo_Nombre', 'Carrera_Nombre'])['Num_Estudiantes'].sum().unstack(fill_value=0)
    print(total_by_period_career)

    print("\n--- Estudiantes por Periodo, Carrera y Semestre (Primeros 5 registros) ---")
    print(full_df.head()) # Muestra los primeros registros del DataFrame completo

    print("\n--- Distribución de Estudiantes por Semestre (Promedio en todos los periodos) ---")
    avg_by_sem = full_df.groupby('Semestre')['Num_Estudiantes'].mean().sort_index()
    print(avg_by_sem)

    print("\n--- TOP 5 Carreras con más estudiantes (Promedio por periodo) ---")
    avg_by_career = full_df.groupby('Carrera_Nombre')['Num_Estudiantes'].sum().sort_values(ascending=False)
    print(avg_by_career.head())
    
    print("\n--- Estudiantes por Tipo de Periodo (A, B, V) ---")
    students_by_period_type = full_df.groupby('Tipo_Periodo')['Num_Estudiantes'].sum()
    print(students_by_period_type)

    return full_df

# --- Ejecutar los análisis ---
df_first_sem = analyze_first_sem_population(all_period_data, period_map, period_type, career_names)
print("\n" + "-"*80 + "\n") # Separador
df_tenth_sem = analyze_tenth_sem_population(all_period_data, period_map, period_type, career_names)
print("\n" + "-"*80 + "\n") # Separador
df_full_exploratory = exploratory_population_analysis(all_period_data, period_map, period_type, career_names)

print("\nAnálisis de población completado.")

Cargando datos desde: D:/TesisDB/CSV's
Cargado: periodo35carrera10semestre1.csv en p35['p35c10s1']
Cargado: periodo35carrera10semestre3.csv en p35['p35c10s3']
Cargado: periodo35carrera10semestre5.csv en p35['p35c10s5']
Cargado: periodo35carrera10semestre7.csv en p35['p35c10s7']
Cargado: periodo35carrera10semestre9.csv en p35['p35c10s9']
Cargado: periodo35carrera11semestre1.csv en p35['p35c11s1']
Cargado: periodo35carrera11semestre3.csv en p35['p35c11s3']
Cargado: periodo35carrera2semestre1.csv en p35['p35c2s1']
Cargado: periodo35carrera2semestre3.csv en p35['p35c2s3']
Cargado: periodo35carrera2semestre5.csv en p35['p35c2s5']
Cargado: periodo35carrera2semestre7.csv en p35['p35c2s7']
Cargado: periodo35carrera2semestre9.csv en p35['p35c2s9']
Cargado: periodo35carrera3semestre1.csv en p35['p35c3s1']
Cargado: periodo35carrera3semestre3.csv en p35['p35c3s3']
Cargado: periodo35carrera3semestre5.csv en p35['p35c3s5']
Cargado: periodo35carrera3semestre7.csv en p35['p35c3s7']
Cargado: periodo35c